<a href="https://colab.research.google.com/github/tirtha0809/Machine-Learning/blob/main/csvcnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from keras.layers import Dense, Conv3D, Flatten, Dropout, MaxPooling3D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [68]:


# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Load the CSV file
try:
    df = pd.read_csv('/content/drive/MyDrive/LANDSLIDE/LandslideData2.csv')
except FileNotFoundError:
    print("Error: CSV file not found. Please check the file path.")
    exit()
except Exception as e:
    print(f"Error occurred while loading CSV file: {e}")
    exit()

# Data Exploration
print(df.head())  # Display the first few rows of the DataFrame
print(df.info())  # Get information about column names, data types, and missing values



# Encode categorical variables
# Note: Assuming 'category' is the target column
X = df.drop('landslide_trigger', axis=1)
y = df['landslide_trigger']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape the data for the 2D CNN model
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1], 1)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   Index  landslide_category  landslide_size  fatality_count  Latitude  \
0   1858                   7               2               0     16.92   
1   1141                   6               2               4     30.38   
2    689                  10               2               0     37.41   
3   2056                   6               2               1      4.58   
4     55                   6               1               0     42.97   

   Longitude  temp-2  temp-1  temp-0  maxt-2  ...  avgslope  LC_Type1_mean  \
0      73.59    26.3    26.5    26.2    27.7  ...    27.875            9.0   
1      78.09    26.7    27.2    25.8    28.1  ...    81.375            9.0   
2     -82.44    14.3    17.8    20.1    18.7  ...    90.125            8.0   
3     101.39    24.5    25.8    26.2    26.0  ...    86.625            9.0   
4    -124.01    11.8     8.3     7.8

In [69]:
print(X_train)

[[[[-0.74234902]
   [-0.21484918]
   [-0.36395779]
   ...
   [-0.4492186 ]
   [ 0.61418462]
   [-1.16224696]]]


 [[[ 1.41618947]
   [-3.57392856]
   [ 1.3854836 ]
   ...
   [-0.25493262]
   [-0.5872355 ]
   [-1.0111159 ]]]


 [[[ 1.03827582]
   [-0.21484918]
   [ 1.3854836 ]
   ...
   [-0.72201741]
   [-0.18538243]
   [-0.92199992]]]


 ...


 [[[ 0.38409427]
   [-0.21484918]
   [-0.36395779]
   ...
   [ 1.71986463]
   [ 0.82392283]
   [ 1.48346104]]]


 [[[ 0.10469881]
   [-0.21484918]
   [-0.36395779]
   ...
   [-0.51176272]
   [-0.53200602]
   [ 0.42335043]]]


 [[[ 0.28401231]
   [-0.21484918]
   [-0.36395779]
   ...
   [ 0.19618347]
   [ 0.26697454]
   [ 1.47417697]]]]


In [70]:
input = tf.keras.Input(shape = (5304, 1, 113, 1))

In [71]:
tensor = tf.convert_to_tensor(X_train)

In [72]:
tensor.shape

TensorShape([5297, 1, 113, 1])

In [73]:
import numpy as np

# Reshape input data to include channel dimension
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

# Verify the shape after reshaping
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)


Shape of X_train: (5297, 1, 113, 1, 1)
Shape of X_test: (1325, 1, 113, 1, 1)


In [80]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

# Reshape the input data to match the expected shape for Conv1D
X_train = X_train.reshape(X_train.shape[0], X_train.shape[2], X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0], X_test.shape[2], X_test.shape[1])

# Define the input shape excluding the batch size dimension
input_shape = X_train.shape[1:]

# Create the model
model = Sequential()
model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(14, activation='softmax'))  # Adjust num_classes based on your problem

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()


Model: "sequential_29"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_18 (Conv1D)          (None, 111, 32)           128       
                                                                 
 max_pooling1d_13 (MaxPooli  (None, 55, 32)            0         
 ng1D)                                                           
                                                                 
 conv1d_19 (Conv1D)          (None, 53, 64)            6208      
                                                                 
 max_pooling1d_14 (MaxPooli  (None, 26, 64)            0         
 ng1D)                                                           
                                                                 
 flatten_6 (Flatten)         (None, 1664)              0         
                                                                 
 dense_12 (Dense)            (None, 128)             

In [81]:
print("Shape of y_train:", y_train.shape)

Shape of y_train: (5297, 15)


In [82]:
from keras.utils import to_categorical

y_train = to_categorical(y_train)

In [83]:
num_classes_train = len(np.unique(y_train))
num_classes_test = len(np.unique(y_test))

print(f"Number of classes in y_train: {num_classes_train}")
print(f"Number of classes in y_test: {num_classes_test}")

Number of classes in y_train: 2
Number of classes in y_test: 13


In [84]:
# prompt: train and evaluate

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print("Loss:", loss)
print("Accuracy:", accuracy)


Epoch 1/10


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/compile_utils.py", line 277, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    